# Simulations on a hollow-sphere domain

The following notebook contains sample code for simulating perfusion and gas exchange on a hollow-sphere mimicking an idealized alveolus.

First we import necessary libraries and add important directories to the system path.

In [ ]:
import sys
import os
import dolfin
import pyvista as pv
import matplotlib.pyplot as plt
plt.style.use('ggplot')
sys.path.append(os.getcwd()[:-6])

Following that, we import the important modules from the model source files.

In [ ]:
from src.model import PerfusionGasExchangeModel
from src.params import params

Now we outline the folder name and the path to save raw data files.

In [ ]:
folder = "sphere-notebook"
path = os.path.join("../raw-data", folder)

Now we run the solver. First, we instance the model. Then, we import the spherical mesh from an XML file. Afterwards, we solve the perfusion problem on the mesh and then we solve the transport problem _without_ hemoglobin effects. Finally, we run the actual full model using the previous solution as an initial guess for the Newton iteration, and save the boundary data.

In [ ]:
model = PerfusionGasExchangeModel(folder_path=path, params=params)
model.import_mesh(os.path.join("../raw-data/meshes", "sphere.xml"), type="xml")
model.sim_p(save=True, meshtype="sphere")
boundaries = dolfin.File(model.folder_path+'/bnd/bnd.pvd')
boundaries << model.boundaries
x = model.sim_sbst(hb=False, save=False)
solution = model.sim_sbst(hb=True, save=True, guess=x)

The preceeding code will generate PVD (Paraview) and VTK files that hold the solutions to our problems. We now visualize the results from a parallel simulation (see the `cluster` directory for details on how to run in a cluster).

In [ ]:
mesh = pv.read(os.path.join('../raw-data','sphere-job','t','pO2.pvd'))
mesh.plot(jupyter_backend='pythreejs')